In [2]:
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:reporting-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-16.0")
user

UserTO(id=9ab40120-eeae-4744-93b3-16ef13dcd0aa, username=Johann-16.0)

In [3]:
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

In [4]:
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.fund.api.model.FundName

val fundSdk = FundSdk()
val expenseFund = runBlocking { fundSdk.getFundByName(user.id, FundName("Expenses")) }
expenseFund

FundTO(id=b607504a-72b4-429e-b83a-9b93ce371487, name=Expenses)

In [5]:
import ro.jf.funds.reporting.api.model.*
import kotlinx.serialization.json.Json
import com.charleskorn.kaml.Yaml

fun getReportDataConfiguration(): ReportDataConfigurationTO {
    val dataConfigurationRawJson: String = File(REPORT_DATA_CONFIGURATION_YAML_FILE).readText()
    val dataConfiguration = Yaml.default.decodeFromString(ReportDataConfigurationTO.serializer(), dataConfigurationRawJson)
    return dataConfiguration
}
val dataConfiguration = getReportDataConfiguration()
dataConfiguration

ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=[ReportGroupTO(name=basic, filter=RecordFilterTO(labels=[basic])), ReportGroupTO(name=home, filter=RecordFilterTO(labels=[home])), ReportGroupTO(name=shopping_services, filter=RecordFilterTO(labels=[shopping_services])), ReportGroupTO(name=transport, filter=RecordFilterTO(labels=[transport])), ReportGroupTO(name=fun, filter=RecordFilterTO(labels=[fun])), ReportGroupTO(name=gifts, filter=RecordFilterTO(labels=[gifts])), ReportGroupTO(name=development, filter=RecordFilterTO(labels=[development]))], features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=true), groupedBudget=GroupedBudgetReportFeatureTO(enabled=true, distributions=[BudgetDistributionTO(default=true, from=null, groups=[GroupBudget

In [6]:
import ro.jf.funds.reporting.sdk.ReportingSdk
import kotlinx.coroutines.delay
import ro.jf.funds.commons.model.labelsOf
import ro.jf.funds.commons.model.Currency
import java.time.Instant

val reportingSdk = ReportingSdk()
val reportView = runBlocking {
    val reportViewName = "Expense report"
    val existingReportView = reportingSdk.listReportViews(user.id).items.firstOrNull { it.name == reportViewName }
    if (existingReportView != null) {
        return@runBlocking existingReportView
    }
    val request = CreateReportViewTO(reportViewName, expenseFund.id, dataConfiguration)
    var task: ReportViewTaskTO = reportingSdk.createReportView(user.id, request)
    val timeout = Instant.now().plusSeconds(120)
    while (task is ReportViewTaskTO.InProgress && Instant.now().isBefore(timeout)) {
        delay(2000)
        task = reportingSdk.getReportViewTask(user.id, task.taskId)
    }
    if (task is ReportViewTaskTO.Completed) {
        task.report
    } else if (task is ReportViewTaskTO.Failed) {
        throw IllegalStateException("Report view creation failed on task $task")
    } else {
        throw IllegalStateException("Report view creation timed out on task $task")
    }
}
reportView

ReportViewTO(id=19763aa8-9a5d-41bc-a456-9481f5ce0b3a, name=Expense report, fundId=b607504a-72b4-429e-b83a-9b93ce371487, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]))))

In [7]:
import ro.jf.funds.reporting.api.model.*

val granularInterval = GranularDateInterval(
    interval = DateInterval(
        LocalDate.parse("2019-01-01"),
        LocalDate.parse("2020-12-31")
    ),
    granularity = TimeGranularity.MONTHLY
)
val report = runBlocking {
    reportingSdk.getReportViewData(user.id, reportView.id, granularInterval)
}
report

ReportDataTO(viewId=19763aa8-9a5d-41bc-a456-9481f5ce0b3a, granularInterval=GranularDateInterval(interval=DateInterval(from=2019-01-01, to=2020-12-31), granularity=MONTHLY), data=[ReportDataItemTO(timeBucket=DateInterval(from=2019-01-01, to=2019-01-31), net=-4082.94, value=ValueReportTO(start=0.0, end=2481.52, min=0.0, max=0.0), groups=[ReportDataGroupItemTO(group=basic, net=-2305.04, allocated=656.446, left=-1648.5936695498056), ReportDataGroupItemTO(group=home, net=-500.0, allocated=656.446, left=156.4466402721593), ReportDataGroupItemTO(group=shopping_services, net=0.0, allocated=656.446, left=656.4457963498469), ReportDataGroupItemTO(group=transport, net=-204.24, allocated=656.446, left=452.2059896389564), ReportDataGroupItemTO(group=fun, net=-1031.16, allocated=656.446, left=-374.7138122918958), ReportDataGroupItemTO(group=gifts, net=0.0, allocated=656.446, left=656.4457963498469), ReportDataGroupItemTO(group=development, net=-42.5, allocated=2625.784, left=2583.2826803033468)]), R

In [8]:
import java.math.BigDecimal
import org.jetbrains.kotlinx.kandy.dsl.* // For creating visualizations
import org.jetbrains.letsPlot.scale.scaleYContinuous

data class MonthlyExpense(val month: LocalDate, val net: Double)

val monthlyReportDF = report.data
    .mapNotNull { reportDataItem ->
        reportDataItem.net?.let { amount ->
        MonthlyExpense(reportDataItem.timeBucket.from, amount.negate().toDouble())
            }
    }
    .toDataFrame()

plot(monthlyReportDF) {
    x("month") {

    }
    bars {
        y("net")

        fillColor = Color.ORANGE
    }
    layout {
        title = "Monthly expenses"
        size = 1800 to 1000
    }
    line {
        y.constant(0)
    }
    line {
        y.constant(10000)
    }
    line {
        y.constant(20000)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="aPoxan"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Monthly expenses"
},
"mapping":{
},
"data":{
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"net":[4082.94,4301.32,8561.68,5486.66,8106.58,4529.1,10769.4,25034.218092,4454.4584992,6433.93,9787.68,5529.22,4552.19,5442.03,34488.16,17363.44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":1800.0,
"height":1000.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"net"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"fill":"#fc8452",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":10000.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month"
},
"stat":"identity",
"sampling":"none",
"y":20000.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"month"
},{
"type":"float",
"column":"net"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("aPoxan");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 1800.0,
 height: 1000.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M104.92193670801498 921.0 L104.92193670801498 921.0 L171.49131939217477 921.0 L231.61850375206996 921.0 L298.187886436237 921.0 L362.6098696789777 921.0 L429.17925236314477 921.0 L493.60123560588545 921.0 L560.1706182900452 921.0 L626.7400009742123 921.0 L691.161984216953 921.0 L757.73136690112 921.0 L822.1533501438607 921.0 L888.7227328280278 921.0 L955.2921155121949 921.0 L1017.5666993135092 921.0 L1084.1360819976762 921.0 L1148.558065240417 921.0 L1215.127447924584 921.0 L1279.5494311673247 921.0 L1346.1188138514917 921.0 L1412.6881965356588 921.0 L1477.1101797783995 921.0 L1543.6795624625593 921.0 L1608.1015457053072 921.0 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(71,71,71)" stroke-opacity="1.0">
 
 
 
 
 
 <path d="M104.92193670801498 666.6684099288402 L104.92193670801498 666.6684099288402 L171.49131939217477 666.6684099288402 L231.61850375206996 666.6684099288402 L298.187886436237 666.6684099288402 L362.6098696789777 666.6684099288402 L429.17925236314477 666.6684099288402 L493.60123560588545 666.6

In [9]:
val categories = mapOf(
    "basic" to Color.GREEN,
    "home" to Color.PEACH,
    "shopping_services" to Color.LIGHT_GREEN,
    "transport" to Color.BLUE,
    "fun" to Color.ORANGE,
    "gifts" to Color.RED,
    "development" to Color.PURPLE,
)

fun getGroupedExpensesDataFrame(): DataFrame<*> {
    val categoriesNetExtractor: Map<String, (List<ReportDataGroupItemTO>) -> Double> =
        categories
            .keys
            .map { group ->
                group to { groups: List<ReportDataGroupItemTO> ->
                    groups.first { it.group == group }.net?.negate()?.toDouble() ?: 0.0
                }
            }
            .toMap()

    return report.data
        .map { monthlyData ->
            monthlyData.timeBucket.from to categoriesNetExtractor.mapValues { (_, extractor) -> extractor(monthlyData.groups!!) }
        }
        .let {
            dataFrameOf(
                "month" to it.flatMap { (month, values) -> values.map { month } },
                "value" to it.flatMap { (_, values) -> values.map { it.value } },
                "category" to it.flatMap { (_, values) -> values.keys }
            )
        }
}
getGroupedExpensesDataFrame().groupBy("category").plot {
    x("month")
    line {
        y("value")
        color("category") {
            scale = categorical(
                *categories.map { (category, color) -> category to color }.toTypedArray()
            )
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expenses by category"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ciMrUe"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Expenses by category"
},
"mapping":{
},
"data":{
"&merged_groups":["basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","basic","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","home","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","shopping_services","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","transport","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","fun","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","gifts","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development","development"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.575

In [11]:
data class MonthlyExpense(val month: LocalDate, val start: Double, val end: Double)

val df = report.data
    .mapNotNull { item ->
        item.value?.let { valueReport ->
            MonthlyExpense(
                item.timeBucket.from,
                valueReport.start.toDouble(),
                valueReport.end.toDouble()
            )
        }
    }
    .let {
        dataFrameOf(
            "month" to it.map { it.month }.let { it + it },
            "value" to it.map { it.start } + it.map { it.end },
            "type" to List(it.size) { "startValue" } + List(it.size) { "endValue" }
        )
    }

df.groupBy("type").plot {
    x("month")
    line {
        y("value")
        color("type") {
            scale = categorical("startValue" to Color.ORANGE, "endValue" to Color.BLUE)
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Expense fund value"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="nQb2qm"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Expense fund value"
},
"mapping":{
},
"data":{
"&merged_groups":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"type":["startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","startValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue","endValue"],
"value":[0.0,2481.52,5185.08,11301.26,11528.2727350348,10643.764995302,11871.901585538,8638.3980216284,-6550.7530315744,-873.7461008179,4052.2818580708,6037.1321957254,12396.0035787372,18628.0469211958,24175.7814668309,22046.8575483166,24076.3746926345,24072.5638477149,24074.9571444815,24077.8498150402,24074.6979992326,24063.0262381633,24062.6620505464,24064.696638801,2481.52,5185.08,11301.26,11526.0160226864,10643.310981436,11868.2374648476,8637.9027559568,-6550.8393119068,-872.2869408013,4051.7634872547,6037.1198616282,12394.3761693426,18628.6707953371,24174.7828379781,22045.4085328505,24075.5881809199,24072.6945814626,24076.192806606,24076.3465951979,24075.3328839652,24063.0415021324,24066.1443028244,24063.4796873483,24065.6199255963]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#fc8452","#5470c6"],
"limits":["startValue","endValue"]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"value",
"color":"type",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"month",
"group":"&merged_groups"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"type"
},{
"type":"datetime",
"column":"month"
},{
"type":"float",
"column":"value"
},{
"type":"str",
"column":"&merged_groups"
}]
},
"spec_id":"8"
};

In [74]:
val dimensions = mapOf(
    "spent" to Color.RED,
    "allocated" to Color.GREEN,
    "left" to Color.YELLOW,
)
val group = "basic"

val dataFrameX = report.data
    .map { item -> item.timeBucket.from to item.groups!!.single { it.group == group } }
    .let { monthlyData: List<Pair<LocalDate, ReportDataGroupItemTO>> ->
        dataFrameOf(
            "month" to monthlyData.flatMap { monthData -> dimensions.map { monthData.first } },
            "value" to monthlyData.flatMap { monthData ->
                dimensions.map {
                    when (it.key) {
                        "spent" -> monthData.second.net
                        "allocated" -> monthData.second.allocated
                        "left" -> monthData.second.left
                        else -> error("unexpected key")
                    }
                }
            },
            "type" to monthlyData.flatMap { dimensions.map { it.key } }
        )
    }

//val dataFrame = dataFrameOf(
//    "type" to dimensions.keys.map { it },
//    "group" to
//)

//dataFrame.plot {
//    x("month")
////    line {
////        y("value")
////        color("type") {
////            scale = categorical("startValue" to Color.ORANGE, "endValue" to Color.BLUE)
////        }
////    }
//    line {
//        y.constant(0)
//    }
//}

val df = dimensions
    .map { dimension ->
        dataFrameOf(
            "month" to report.data.map { it.timeBucket.from },
            "type" to List(report.data.size) { dimension.key },
            "value" to report.data
                .map { it.groups!!.single { it.group == group } }
                .map { groupData ->
                    when (dimension.key) {
                        "spent" -> groupData.net
                        "allocated" -> groupData.allocated
                        "left" -> groupData.left
                        else -> error("unexpected key")
                    }
                }
        )
    }
    .reduce { a, b -> a.concat(b) }

df.groupBy("type").plot {
    x("month")
    line {
        y("value")
        color("type") {
            scale = categorical(
                *dimensions.map { (category, color) -> category to color }.toTypedArray()
            )
        }
    }
    line {
        y.constant(0)
    }
    layout {
        title = "Basic expenses"
        size = 1200 to 600
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="8cFGRJ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic expenses"
},
"mapping":{
},
"data":{
"&merged_groups":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"month":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12],
"type":["spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","spent","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","allocated","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left","left"],
"value":[-2305.04,-2509.31,-3347.92,-2293.91,-2108.18,-2382.69,-3709.61,-2493.78,-1587.87,-2253.23,-2992.86,-2517.62,-2158.1,-2165.51,-2324.86,-948.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,656.446,700.488,1467.786,571.14160226864,722.82,576.09,753.665,984.681,1013.60535174615,1136.137,1178.232,1188.753,1078.415,1099.955,3236.623,1939.552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1648.5936695498056,-3457.415025517453,-5337.555702209024,-7058.167530131429,-8438.19451757944,-10237.084131886148,-13149.256935838413,-14645.0683052172,-15287.497288387698,-16439.0684049535,-18153.1598703445,-19486.52515355848,-20571.020458615203,-21630.354933571267,-20653.608951363138,-19666.38652219701,-19664.022429404988,-19666.88050598692,-19667.006152437636,-19666.17794264088,-19656.135790081837,-19658.67080187573,-19656.493790740107,-19658.24238168712]
},
"ggsize":{
"width":1200.0,
"height":600.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#ee6666","#3ba272","#fac858"],
"limits":["spent","allocated","left"]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"month",
"y":"value"